# --- Day 9: Movie Theater ---


In [143]:
# --- Support Functions ---
# This section contains support functions used by the main code.

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

def get_area(a:tuple[int,int], b:tuple[int,int]) -> int:
    s1 = abs(a[0] - b[0]) + 1
    s2 = abs(a[1] - b[1]) + 1
    return s1 * s2

points = {}

def point_inside(x,y, h_edges):
    # print(f"point_inside({x},{y})")
    if (x,y) in points:
        return points[(x,y)]
    count = 0
    aligned = 0
    for ey, x1, x2 in h_edges:
        #The point is a border
        if y == ey and x1 <= x <= x2:
            points[(x,y)] = True
            return True
        # the vertical beam traverse a horizontal edge ?
        if ey > y and x1 <= x <= x2:
            # print(f"!!! edge : {ey,x1,x2}")
            if (x == x1):
                if aligned != 2:
                    aligned = 1
                    count += 1
                continue
            if (x == x2):
                if aligned != 1:
                    aligned = 2
                    count += 1
                continue
            count +=1        
            # print(f"   edge {count} : {ey,x1,x2}")
    
    points[(x,y)] = (count % 2) == 1         
    return (count % 2) == 1  # inside if traverse odd edges

def seg_intersect( p1 , p2 , p3, p4) -> bool:
    """
    p1, p2 is segment 1
    p3, p4 is segment 2
    return True for a intersection
    
    Note segments are Horizontal or Vertical
    """ 
    p1, p2 = sorted([p1, p2])
    p3, p4 = sorted([p3, p4])

    is_vert_1 = p1[0] == p2[0]  # same x = vertical = True
    is_vert_2 = p3[0] == p4[0]

    if is_vert_1 == is_vert_2: # parallels do not intersect
        return False

    if is_vert_1:
        xc, y0 = p1
        _, y1 = p2
        x0,yc = p3
        x1, _ = p4
        return x0 <= xc <= x1 and y0 < yc < y1
    else:
        x0,yc = p1
        x1, _ = p2
        xc, y0 = p3
        _, y1 = p4
        return x0 < xc < x1 and y0 <= yc <= y1



def is_area_inside( a:tuple[int,int], b:tuple[int,int], 
                    h_edges:dict[int: list[int]], v_edges:dict[int: list[int]]):
    # check if the sides of rectangle not intersect edges

    # horizontal sides

    for h1, h2 in [((a[0],a[1]),(b[0],a[1])), ((a[0],b[1]),(b[0],b[1]))]:
        for v1, v2 in v_edges:
            if seg_intersect(h1, h2, v1, v2):
                return False
    for v1, v2 in [((a[0],a[1]),(a[0],b[1])), ((b[0],a[1]),(b[0],b[1]))]:
        for h1, h2 in h_edges:
            if seg_intersect(h1, h2, v1, v2):
                return False
    return True


def sign(x):
    return (x > 0) - (x < 0)


In [31]:
# --- Part ONE ---

input = read_input('input.txt')

tiles = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

# print (tiles)

areas = []
for i, a in enumerate(tiles):
    for b in tiles[i+1:]:
        areas.append(get_area(a, b))

#print (areas)
result = max(areas)


print("Part ONE:", result)

Part ONE: 4763040296


In [145]:
# --- Part TWO ---
input = read_input('input.txt')

result = 0

# create the border tiles

red_tiles = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

n = len(red_tiles)

# store all horizontal and vertical edges
h_edges = []
v_edges = []

for i in range(n):
    x1, y1 = red_tiles[i]
    x2, y2 = red_tiles[(i+1) % n]
    if y1 == y2:
        h_edges.append(((x1, y1),(x2,y2)))
    else:
        v_edges.append(((x1, y1),(x2,y2)))

# print (h_edges)  # horizontal edges 
# print (v_edges)  # vertical edges

max_area = 0 
for i, a in enumerate(red_tiles):
    for b in red_tiles[i+1:]:
        area = get_area(a, b)
        if area <= max_area :
            # print(f"SMALL_AREA for {a,b}")
            continue
        if is_area_inside(a, b, h_edges, v_edges):
            print (f'IS_TRUE : {a,b} Area = {area}')               
            max_area = area


# print (h_edges)
result = max_area

print("Part TWO:", result)

IS_TRUE : ((97554, 50097), (97554, 51315)) Area = 1219
IS_TRUE : ((97554, 50097), (2532, 52309)) Area = 210285899
IS_TRUE : ((97454, 54924), (2532, 52309)) Area = 248318568
IS_TRUE : ((97454, 54924), (2532, 51107)) Area = 362416014
IS_TRUE : ((97663, 58643), (83709, 84872)) Area = 366039650
IS_TRUE : ((97663, 58643), (83012, 85506)) Area = 393611328
IS_TRUE : ((97663, 58643), (81946, 86280)) Area = 434414084
IS_TRUE : ((97663, 58643), (81016, 87071)) Area = 473285992
IS_TRUE : ((97663, 58643), (80101, 88296)) Area = 520813202
IS_TRUE : ((97663, 58643), (79517, 88974)) Area = 550434804
IS_TRUE : ((97663, 58643), (78496, 89787)) Area = 596987360
IS_TRUE : ((97663, 58643), (77571, 89907)) Area = 628207645
IS_TRUE : ((97663, 58643), (76184, 90672)) Area = 688004400
IS_TRUE : ((97663, 58643), (75235, 91328)) Area = 733114294
IS_TRUE : ((97663, 58643), (74214, 92452)) Area = 792844500
IS_TRUE : ((97663, 58643), (73448, 92967)) Area = 831214200
IS_TRUE : ((97663, 58643), (72332, 92998)) Area 

4634026886

In [146]:
def point_in_polygon(x, y, polygon):
    """
    polygon: lista di tuple (x, y)
    ritorna True se il punto è dentro il poligono, False altrimenti
    """
    inside = False
    n = len(polygon)

    for i in range(n):
        x1, y1 = polygon[i]
        x2, y2 = polygon[(i + 1) % n]

        # Controlla se il punto è tra le y dei due vertici
        intersects = ((y1 > y) != (y2 > y)) and \
                     (x < (x2 - x1) * (y - y1) / (y2 - y1 + 1e-12) + x1)

        if intersects:
            inside = not inside

    return inside

def seg_intersect(p1, p2, p3, p4):
    # p1,p2 definiscono segmento 1; p3,p4 segmento 2.
    def orient(a, b, c):
        return (b[0] - a[0])*(c[1] - a[1]) - (b[1] - a[1])*(c[0] - a[0])

    def on_seg(a, b, c):
        return (min(a[0], b[0]) <= c[0] <= max(a[0], b[0]) and
                min(a[1], b[1]) <= c[1] <= max(a[1], b[1]))

    o1 = orient(p1, p2, p3)
    o2 = orient(p1, p2, p4)
    o3 = orient(p3, p4, p1)
    o4 = orient(p3, p4, p2)

    # Generico caso di intersezione
    if o1*o2 < 0 and o3*o4 < 0:
        return True

    # Casi speciali (collineare)
    if o1 == 0 and on_seg(p1, p2, p3): return True
    if o2 == 0 and on_seg(p1, p2, p4): return True
    if o3 == 0 and on_seg(p3, p4, p1): return True
    if o4 == 0 and on_seg(p3, p4, p2): return True

    return False

def rect_inside_polygon(rect, polygon):
    # rect = [(x1,y1),(x2,y2)]
    # ordino
    (x1,y1),(x2,y2) = rect
    xmin, xmax = sorted([x1,x2])
    ymin, ymax = sorted([y1,y2])

    # i 4 vertici del rettangolo
    R = [
        (xmin,ymin),
        (xmin,ymax),
        (xmax,ymin),
        (xmax,ymax)
    ]

    # 1. Tutti vertici dentro o sul bordo
    for (xr, yr) in R:
        state = point_in_polygon(xr, yr, polygon)
        if state == "outside":
            return False

    # 2. Nessun lato interseca il perimetro
    rect_edges = [
        ((xmin,ymin), (xmin,ymax)),  # sinistro
        ((xmax,ymin), (xmax,ymax)),  # destro
        ((xmin,ymin), (xmax,ymin)),  # basso
        ((xmin,ymax), (xmax,ymax))   # alto
    ]

    n = len(polygon)
    poly_edges = [
        (polygon[i], polygon[(i+1)%n])
        for i in range(n)
    ]

    for e1 in rect_edges:
        for e2 in poly_edges:
            if seg_intersect(e1[0], e1[1], e2[0], e2[1]):
                return False

    return True

input = read_input('input.txt')

result = 0

# create the border tiles

polygon = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

max_area = 0 
for i, a in enumerate(red_tiles):
    for b in red_tiles[i+1:]:
        area = get_area(a, b)
        if area <= max_area :
            # print(f"SMALL_AREA for {a,b}")
            continue
        if rect_inside_polygon([a, b], polygon):
            print (f'IS_TRUE : {a,b} Area = {area}')               
            max_area = area


# print (h_edges)
result = max_area

print("Part TWO:", result)

Part TWO: 0
